# Web Scraping Hotel and Review Data

### Import Libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib as plt
import json

In [2]:
import requests
import bs4
from bs4 import BeautifulSoup
from tqdm import tqdm
import time
import re

In [3]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from time import time, sleep

In [6]:
import os

### Sections
1.0 Hotel Links
- 1.1 [Code for scraping hotel data links for London Hotels](#scrape_links)
- 1.2 [Save link data to CSV file](#link_save)
- 1.3 [Load link data from CSV file](#link_load)

2.0 Functions to extract data
- 2.1 [Hotel data functions](#hotel_func)
- 2.2 [Review data functions](#review_func)

3.0 Scraping Hotel Data
- 3.1 [Code to scrape hotel data](#hotel_scrape)
- 3.2 [Load hotel data from CSV](#hotel_load)
- 3.2.1 [Add approximate time to scrape reviews column](#hotel_tts)

4.0 Scraping Review Data
- 4.1 [Code to scrape review data](#review_scrape)
- 4.2 [Load review data from CSV](#review_load)

## 1.0 Hotel Links

<a name='scrape_links'></a>
### 1.1 Code for scraping hotel data links for London Hotels

In [38]:
# Set up initial scrape to get all London hotel links from website
status = []
links = []
for i in range(0, 900, 900):
    for page in tqdm(range(3600,3900,30)):
        sleep(np.random.randint(25,45))
        try:
            r = requests.get(f'https://www.tripadvisor.co.uk/Hotels-g186338-oa{page}-London_England-Hotels.html')
        except:
            print('page not found')
        status.append(r.status_code)
        soup = BeautifulSoup(r.text, 'html.parser')
        
        try: 
            links = links + [link['href'] for link in soup.find_all('a', attrs={"class":"review_count"})]
            sleep(np.random.randint(25,45))
        except:
            print('error with links')
            sleep(np.random.randint(100,200))
    
        print(page, len(links))
    
    print('big sleep')
    #sleep(np.random.randint(100,150))

 10%|█         | 1/10 [01:16<11:28, 76.52s/it]

3600 1890


 20%|██        | 2/10 [02:34<10:18, 77.33s/it]

3630 1920


 30%|███       | 3/10 [03:39<08:22, 71.84s/it]

...

3810 1980
error with links


 90%|█████████ | 9/10 [20:10<02:46, 166.55s/it]

3840 1980
error with links


100%|██████████| 10/10 [23:41<00:00, 142.15s/it]

3870 1980
big sleep


<a name='link_save'></a>
### 1.2 Save link data to CSV file

In [17]:
# save links to csv
# df = pd.DataFrame(links, columns=['links'])
# df.to_csv(f'./TripAdvisor_ReviewLinks_London_')

<a name='link_load'></a>
### 1.3 Load link data from CSV file

In [12]:
# load links csv into dataframe
links = [link for link in os.listdir() if 'Links' in link]
df_links = pd.DataFrame()
for link in links:
    df_link = pd.read_csv(link)
    df_links = pd.concat([df_links, df_link], ignore_index=True)

# drop duplicates
df_links = df_links.drop('Unnamed: 0', axis=1).drop_duplicates(ignore_index=True)

## 2.0 Functions to extract data

<a name='hotel_func'></a>
### 2.1 Hotel data functions

In [ ]:
# extract html from an hotel to test functions
url_review = 'https://www.tripadvisor.co.uk/Hotel_Review-g186338-d193660-Reviews-London_Marriott_Hotel_Kensington-London_England.html'

r_review = requests.get(url_review)
soup_review = BeautifulSoup(r_review.text, 'html.parser')

In [ ]:
# get the hotel name
def get_hotel_name(result):
    try:
        return result.find('h1').text
    except:
        return 'None'

In [22]:
get_hotel_name(soup_review)

'London Marriott Hotel Kensington Reviews'

In [ ]:
# get the average score
def get_hotel_avg_score(result):
    try:
        return result.find(class_='_3cjYfwwQ').text
    except:
        return 'None'

In [ ]:
get_hotel_avg_score(soup_review)

In [25]:
# get the total number of reviews for hotel
def get_hotel_num_reviews(result):
    try:
        return int(result.find(class_='_3jEYFo-z').text.split(' ')[0].replace(',', ''))
    except:
        return 'None'

In [26]:
get_hotel_num_reviews(soup_review)

1942

In [27]:
# get the hotel review number (may/may not be useful)
def get_hotel_review_number(result):
    try:
        return result.find('span', class_='_28eYYeHH').text
    except:
        return 'None'

In [28]:
get_hotel_review_number(soup_review)

'#415 of 1,173 hotels in London'

In [29]:
# get the breakdon of the review score, returns dictionary
def get_hotel_avg_score_breakdown(result):
    try:
        return {div.text: int(div.span['class'][1][-2:])/10 for div in result.find_all('div', class_='_1krg1t5y')}
    except:
        return 'None'

In [30]:
get_hotel_avg_score_breakdown(soup_review)

{'Location': 4.5, 'Cleanliness': 4.5, 'Service': 4.5, 'Value': 4.0}

In [31]:
# extract the hotel description
def get_hotel_desc(result):
    try:
        return result.find('div', class_='cPQsENeY').text
    except:
        return 'None'

In [32]:
get_hotel_desc(soup_review)

"Discover sophisticated comfort at London Marriott Hotel Kensington. Set in an ideal location, our hotel in Kensington, England offers easy access to London's most celebrated landmarks, including Hyde Park, the Natural History Museum and Kensington Palace."

In [38]:
# extract the hotel "style" information, returns data as a list
def get_hotel_style(result):
    try:
        return [div.get_text() for div in result.find_all('div', class_='_2dtF3ueh')]
    except:
        return 'None'

In [39]:
get_hotel_style(soup_review)

['', 'Green', 'Business']

In [40]:
# get the hotel star rating
def get_hotel_star_rating(result):
    try:
        return float(result.find('svg', class_='AZd6Ff4E')['title'].split(' ')[0])
    except:
        return 'None'

In [41]:
get_hotel_star_rating(soup_review)

4.0

In [42]:
# get the hotel location details, returns data in a list
def get_hotel_location_details(result):
    try:
        return [div.get_text() for div in result.find_all('div', class_='_27i_rL5x ui_column is-4')]
    except:
        return 'None'

In [43]:
get_hotel_location_details(soup_review)

['100Great for walkersGrade: 100 out of 100',
 '104Restaurantswithin 0.3 miles',
 '8Attractionswithin 0.3 miles']

In [44]:
# extract the address
def get_hotel_address(result):
    try:
        return result.find(class_='_1sPw_t0w _2oAeY-0w').text
    except:
        return 'None'

In [45]:
get_hotel_address(soup_review)

'147C Cromwell Road Kensington, London SW5 0TH England'

In [46]:
# extract details of hotel amenities
def get_hotel_amenities(result):
    try:
        return ', '.join([div.text for div in result.find_all(class_='_2rdvbNSg')])
    except:
        return 'None'

In [47]:
get_hotel_amenities(soup_review)

'Paid private parking on-site, Wifi, Fitness Centre with Gym / Workout Room, Bar / lounge, Car hire, Conference facilities, Meeting rooms, Baggage storage, Secured parking, Paid wifi, Restaurant, Breakfast buffet, Breakfast in the room, Complimentary instant cofffee, Concierge, Currency exchange, Newspaper, Non-smoking hotel, ATM on site, 24-hour front desk, Express check-in / check-out, Dry cleaning, Allergy-free room, Air conditioning, Housekeeping, Room service, Safe, VIP room facilities, Flatscreen TV, Non-smoking rooms, Suites, Family rooms'

In [48]:
# get the overall number of reviews split by rating - list ordered [5, 4, 3, 2, 1]
def get_hotel_overall_review_ratings(result):
    try:
        return [int(span.text) for span in result.find_all(class_='_3fVK8yi6')]
    except:
        return 'None'

In [49]:
get_hotel_overall_review_ratings(soup_review)

[796, 704, 295, 89, 58]

In [51]:
# get the total number of reviews split by language, returns dictionary
def get_hotel_top_language_reviews(result):
    try:
        return dict(zip([i.text for i in result.find_all(class_='_1wk-I7LS')], 
                        [int(i.text.replace('(','').replace(')','').replace(',', '')) for i in result.find_all(class_='mxlinKbW')]))
    except:
        return 'None'

In [52]:
get_hotel_top_language_reviews(soup_review)

{'All languages': 1942,
 'English': 1774,
 'Chinese (Sim.)': 140,
 'Chinese (Trad.)': 130}

In [53]:
# miscellaneous extra details including average room rates, returns data as a list
def get_hotel_extra_details(result):
    try:
        return [i.text for i in result.find_all(class_='_1NHwuRzF')]
    except:
        return 'None'

In [54]:
get_hotel_extra_details(soup_review)

['£119 - £253 (Based on Average Rates for a Standard Room) ',
 'marriott london, london marriott',
 'United KingdomEnglandLondonEarls Court / South Kensington',
 '226']

<a name='review_func'></a>
### 2.2 Review data functions

In [55]:
# get all reviews on page (maximum of 5 per page) and return as list
def get_reviews(result):
    try:
        return result.find_all(class_='_2wrUUKlw _3hFEdNs8')
    except:
        return 'None'

In [67]:
len(get_reviews(soup_review))

5

In [57]:
# function to extract review headline from review list
def review_headline(result):
    try:
        return result.find(class_='glasR4aX').text
    except:
        return 'None'

In [58]:
review_headline(get_reviews(soup_review)[0])

'Great'

In [60]:
# function to extract main body of review from review list
def review(result):
    try:
        return result.find(class_='IRsGHoPm').text
    except:
        return 'None'

In [68]:
review(get_reviews(soup_review)[0])

'The team really couldn’t have done more for us. Lovely welcome. Our room was nice and clean all amenities spot on Marriott quality. All Covid protocols were followed very well and we felt safe.  Breakfast was great . At checkout the team were very helpful and again above abs beyond. . We look forward to returning'

In [69]:
# function to extract review date from review list
def review_date(result):
    try:
        return result.find(class_='_34Xs-BQm').text
    except:
        return 'None'

In [70]:
review_date(get_reviews(soup_review)[0])

'Date of stay: May 2021'

In [71]:
# function to extract review score from review list
def review_score(result):
    try:
        return int(result.find(class_='nf9vGX55').span['class'][1][-2:])/10
    except:
        return 'None'

In [72]:
review_score(get_reviews(soup_review)[0])

5.0

## 3.0 Scraping Hotel Data

In [78]:
# total number of hotels to scrape
df = df_links.copy()
len(df.links)

1924

<a name='hotel_scrape'></a>
### 3.1 Code to scrape hotel data

In [120]:
# iterate through each hotel link and extract hotel details using functions above and store results in dictionary
status = []

# set-up blank hotel data dictionary
hotel_dict = {'link':[],
            'hotel_name':[],
             'address': [],
              'star_rating': [],
              'avg_review_score': [],
              'avg_review_score_breakdown': [],              
              'hotel_description': [],
              'hotel_style': [],
             'location_details': [],
              'location_amenities': [],
             'hotel_review_number': [],
             'num_reviews': [],
             'num_review_breakdown': [],
             'top_lang_reviews': [],
              'extra_details': [],
             }

for i in range(0, 900, 900):
    for link in tqdm(df.links[1000:1924]):
        sleep(np.random.randint(5, 30))#(25,45))
        try:
            r = requests.get(f'https://www.tripadvisor.co.uk/{link}')
        except:
            print('page not found')
        status.append(r.status_code)
        soup = BeautifulSoup(r.text, 'html.parser')
        
        try:
            hotel_dict['link'].append(link)
            hotel_dict['hotel_name'].append(get_hotel_name(soup))
            hotel_dict['address'].append(get_hotel_address(soup))
            hotel_dict['star_rating'].append(get_hotel_star_rating(soup))
            hotel_dict['avg_review_score'].append(get_hotel_avg_score(soup))
            hotel_dict['avg_review_score_breakdown'].append(get_hotel_avg_score_breakdown(soup))
            hotel_dict['hotel_description'].append(get_hotel_desc(soup))
            hotel_dict['hotel_style'].append(get_hotel_style(soup))
            hotel_dict['location_details'].append(get_hotel_location_details(soup))
            hotel_dict['location_amenities'].append(get_hotel_amenities(soup))
            hotel_dict['hotel_review_number'].append(get_hotel_review_number(soup))
            hotel_dict['num_reviews'].append(get_hotel_num_reviews(soup))
            hotel_dict['num_review_breakdown'].append(get_hotel_overall_review_ratings(soup))
            hotel_dict['top_lang_reviews'].append(get_hotel_top_language_reviews(soup))
            hotel_dict['extra_details'].append(get_hotel_extra_details(soup))
            #sleep(np.random.randint(15,45))
        except:
            print('error with links')
            sleep(np.random.randint(100,200))
        
        # save to csv file
        df_hotel_dict = pd.DataFrame(hotel_dict)
        df_hotel_dict.to_csv(f'./TripAdvisor_HotelDetails_London_20210605_400_1924')
        print(link, hotel_dict['hotel_name'][-1])
    
    print('big sleep')
    #sleep(np.random.randint(100,150))

  0%|          | 1/924 [00:15<3:55:44, 15.32s/it]

/Hotel_Review-g186338-d288609-Reviews-Sundial_Court-London_England.html#REVIEWS Sundial Court Hostel Reviews, London


  0%|          | 2/924 [00:29<3:44:25, 14.61s/it]

/Hotel_Review-g186338-d14053698-Reviews-A_L_Lodgings-London_England.html#REVIEWS A & L Lodgings Guesthouse Reviews, London


  0%|          | 3/924 [00:43<3:40:17, 14.35s/it]

...

/Hotel_Review-g186338-d7728390-Reviews-Karslake_Rooms-London_England.html#REVIEWS Karslake Rooms Apartment Reviews, London


100%|█████████▉| 923/924 [4:44:05<00:17, 17.12s/it]

/Hotel_Review-g186338-d6212241-Reviews-Holland_Apartments-London_England.html#REVIEWS Holland Apartments Reviews, London


100%|██████████| 924/924 [4:44:21<00:00, 18.46s/it]

/Hotel_Review-g186338-d3221483-Reviews-Zenith_Hostel-London_England.html#REVIEWS Zenith Hostel Reviews, London
big sleep


<a name='hotel_load'></a>
### 3.2 Load hotel data from CSV

In [5]:
# extract scrapped hotel dict data and store in dataframe
hotel_datasets = [file for file in os.listdir() if 'Details_' in file]
df_hotel_dict = pd.DataFrame()
for dataset in hotel_datasets:
    df_dataset = pd.read_csv(dataset)
    df_hotel_dict = pd.concat([df_hotel_dict, df_dataset], ignore_index=True)

<a name='hotel_tts'></a>
### 3.2.1 Add approximate time to scrape reviews column

In [6]:
# create num english reviews count and approximate time to scrape reviews column
df_hotel_dict['num_reviews_english'] = df_hotel_dict.top_lang_reviews.apply(lambda x: eval(x)['English'])
df_hotel_dict['approx_time_scrape_hrs'] = (df_hotel_dict.num_reviews_english / 5 * 20)/3600

In [7]:
df_hotel_dict.head()

,Unnamed: 0,link,hotel_name,address,star_rating,avg_review_score,avg_review_score_breakdown,hotel_description,hotel_style,location_details,location_amenities,hotel_review_number,num_reviews,num_review_breakdown,top_lang_reviews,extra_details,num_reviews_english,approx_time_scrape_hrs
0,0,/Hotel_Review-g186338-d1371050-Reviews-The_Ken...,"The Kensington Hotel Reviews, London","109-113 Queen's Gate South Kensington, London ...",4.0,4.5,"{'Location': 5.0, 'Cleanliness': 5.0, 'Service...",Traditional service and Victorian grandeur com...,"['', 'Boutique', 'Charming', 'English, French']","['100Great for walkersGrade: 100 out of 100', ...","Free High Speed Internet (WiFi), Fitness Centr...","#16 of 1,167 hotels in London",3753,None,"{'All languages': 3753, 'English': 3419, 'Chin...",['£228 - £356 (Based on Average Rates for a St...,3419,3.798889
1,1,/Hotel_Review-g186338-d187686-Reviews-The_Savo...,"The Savoy Hotel Reviews, London","The Strand, London WC2R 0EZ England",5.0,4.5,"{'Location': 5.0, 'Cleanliness': 5.0, 'Service...","Over the last few months, our team have been w...","['', 'Classic', 'Luxury', 'English, French, It...","['100Great for walkersGrade: 100 out of 100', ...","Paid private parking nearby, Free High Speed I...","#23 of 1,167 hotels in London",6300,None,"{'All languages': 6304, 'English': 5901, 'Chin...",['£367 - £628 (Based on Average Rates for a St...,5901,6.556667
2,2,/Hotel_Review-g186338-d15134890-Reviews-The_Re...,"The Resident Covent Garden Hotel Reviews, London","51 Bedford Street Drift.every.atoms, London WC...",None,5.0,"{'Location': 5.0, 'Cleanliness': 5.0, 'Service...","Renowned for its elegant central Piazza, charm...","['English, Spanish, Arabic, Danish and 5 more']","['100Great for walkersGrade: 100 out of 100', ...","Free High Speed Internet (WiFi), Breakfast ava...","#2 of 1,167 hotels in London",551,"[513, 37, 2, 1, 1]","{'All languages': 554, 'English': 498, 'French...",['£152 - £302 (Based on Average Rates for a St...,498,0.553333
3,3,/Hotel_Review-g186338-d2048553-Reviews-Leonard...,Leonardo Royal London Tower Bridge Hotel Reviews,"45 Prescot Street, London E1 8GP England",4.0,4.5,"{'Location': 4.5, 'Cleanliness': 4.5, 'Service...",Leonardo Royal London Tower Bridge redefines l...,"['', 'English, Polish']","['100Great for walkersGrade: 100 out of 100', ...","Free High Speed Internet (WiFi), Pool, Fitness...","#142 of 1,167 hotels in London",1147,"[766, 198, 86, 52, 49]","{'All languages': 1151, 'English': 1074, 'Fren...",['£86 - £232 (Based on Average Rates for a Sta...,1074,1.193333
4,4,/Hotel_Review-g186338-d6484754-Reviews-Shangri...,"Shangri-La Hotel, At The Shard, London Reviews","31 St Thomas Street, London SE1 9QU England",5.0,4.5,"{'Location': 4.5, 'Cleanliness': 5.0, 'Service...","Feel an unparalleled connection to London, at ...","['', 'City View', 'Great View', 'English, Fren...","['100Great for walkersGrade: 100 out of 100', ...","Valet parking, Free High Speed Internet (WiFi)...","#15 of 1,167 hotels in London",3898,None,"{'All languages': 3903, 'English': 3632, 'Chin...",['£389 - £581 (Based on Average Rates for a St...,3632,4.035556


## 4.0 Scraping Review Data

<a name='review_scrape'></a>
### 4.1 Code to scrape review data

In [140]:
# web scraping review data for each hotel. 


status = []

# loop through each hotel incramentally using index and link in the hotel dictionary scraped data
for i in [788,  789,  761,  791,  778, 1060,  772,  766,  769, 1058,
             747,  765,  776,  768, 1243, 1236,  758,  786,  773]:
    hotel = df_hotel_dict.iloc[i]
    link_list = hotel.link.split('-')
    name = hotel.hotel_name
    num_reviews = eval(hotel.top_lang_reviews)['English']
    
    # set-up review data dictionary
    review_dict = {'link':[],
            'hotel_name':[],
             'review_headline': [],
               'review': [],
               'review_date': [],
               'review_score': []
             }
    
    # each page displays 5 reviews at a time, increment by 5
    for j in tqdm(range(0, num_reviews, 5)):
        link_list.insert(4, f'or{j}')
        link = '-'.join(link_list)
        del link_list[4]
        
        sleep(np.random.randint(5, 30))#(25,45))
        try:
            r = requests.get(f'https://www.tripadvisor.co.uk/{link}')
        except:
            print('page not found')
        status.append(r.status_code)
        soup = BeautifulSoup(r.text, 'html.parser')
        
        try:            
            page_reviews = get_reviews(soup)
            
            for k in range(len(page_reviews)):
                try:
                    review_dict['link'].append(link)
                    review_dict['hotel_name'].append(name)
                    review_dict['review_headline'].append(review_headline(page_reviews[k]))
                    review_dict['review'].append(review(page_reviews[k]))
                    review_dict['review_date'].append(review_date(page_reviews[k]))
                    review_dict['review_score'].append(review_score(page_reviews[k]))

                    #sleep(np.random.randint(15,45))
                except:
                    print('error with review_dict')
                    sleep(np.random.randint(100,200))
        except:
            print('error with for loop')
            sleep(np.random.randint(100,200))
        
        # save data into csv fater each incrementation
        df_review_dict = pd.DataFrame(review_dict)
        df_review_dict.to_csv(f'TripAdvisor_Reviews_London_20210608_{i}_lr_0_{num_reviews}')
        
        print(link, review_dict['review_headline'][-1])
    
    print('big sleep')
    #sleep(np.random.randint(100,150))

  2%|█▋                                                                                 | 1/49 [00:24<19:32, 24.43s/it]

/Hotel_Review-g186338-d1107818-Reviews-or0-Holland_Inn-London_England.html#REVIEWS Not amazing, but not that bad!


  4%|███▍                                                                               | 2/49 [00:51<20:33, 26.25s/it]

/Hotel_Review-g186338-d1107818-Reviews-or5-Holland_Inn-London_England.html#REVIEWS Awful


  6%|█████                                                                              | 3/49 [01:17<19:44, 25.75s/it]

...

/Hotel_Review-g186338-d596248-Reviews-or105-Four_Stars_Hotel-London_England.html#REVIEWS Horrible place- don't come here!


 96%|██████████████████████████████████████████████████████████████████████████████▌   | 23/24 [06:57<00:16, 16.75s/it]

/Hotel_Review-g186338-d596248-Reviews-or110-Four_Stars_Hotel-London_England.html#REVIEWS old but cheap, clean and in a good location!


100%|██████████████████████████████████████████████████████████████████████████████████| 24/24 [07:10<00:00, 17.92s/it]

/Hotel_Review-g186338-d596248-Reviews-or115-Four_Stars_Hotel-London_England.html#REVIEWS Anon
big sleep


<a name='review_load'></a>
### 4.2 Load review data from CSV

In [10]:
# load in review data and save as dataframe
review_datasets = [file for file in os.listdir('data/') if 'Reviews_' in file]
df_review_dict = pd.DataFrame()
for dataset in review_datasets:
    df_dataset = pd.read_csv('data/' + dataset)
    df_review_dict = pd.concat([df_review_dict, df_dataset], ignore_index=True)
    
# drop duplicates
df_review_dict.drop_duplicates(inplace=True)

In [11]:
df_review_dict.head()

,Unnamed: 0,link,hotel_name,review_headline,review,review_date,review_score
0,0,/Hotel_Review-g186338-d1371050-Reviews-or0-The...,"The Kensington Hotel Reviews, London",TheKensington,Amazing hotel professional staff who could not...,Date of stay: May 2021,5.0
1,1,/Hotel_Review-g186338-d1371050-Reviews-or0-The...,"The Kensington Hotel Reviews, London",Love this place,Stayed here as a family of four with two small...,Date of stay: June 2021,5.0
2,2,/Hotel_Review-g186338-d1371050-Reviews-or0-The...,"The Kensington Hotel Reviews, London",Amazing afternoon tea on Saturday 29th May,Our son and his girlfriend treated us to after...,Date of stay: May 2021,5.0
3,3,/Hotel_Review-g186338-d1371050-Reviews-or0-The...,"The Kensington Hotel Reviews, London",A home in London,"This Hotel is first class Mr Tardelli , the Ge...",Date of stay: May 2021,5.0
4,4,/Hotel_Review-g186338-d1371050-Reviews-or0-The...,"The Kensington Hotel Reviews, London",Excellent hotel and great location,I found this hotel on booking.com and was draw...,Date of stay: May 2021,5.0


In [129]:
# total number of reviews
len(df_review_dict)

283624

In [16]:
# reviews split by score
df_review_dict.review_score.value_counts()

5.0    113847
4.0     97743
3.0     47241
1.0     24924
2.0     22024
Name: review_score, dtype: int64

In [17]:
# reviews split by score and normalized. Baseline for data-set is 0.372
df_review_dict.review_score.value_counts(normalize=True)

5.0    0.372318
4.0    0.319652
3.0    0.154494
1.0    0.081510
2.0    0.072026
Name: review_score, dtype: float64